# Team 7 - Garbage DumPY
## Demonstration of Basic Forward and Inverse Kinematics
### MANE 4120/6120 - Robotics 1 - Fall 2023

1. Import RPI Robotics Toolbox, Yahboom Jetson DOFBOT Code, etc, and Initialize Robot Object

In [5]:
import general_robotics_toolbox.general_robotics_toolbox as grt
import general_robotics_toolbox.general_robotics_toolbox_invkin as grt_ik
import Arm, Dofbot
from Arm_Lib import Arm_Device
import time
import numpy as np
import math
import path_generation
import move_read

# Define all the joint lengths [m]
l0 = 61 * 10**-3
l1 = 43.5 * 10**-3
l2 = 82.85 * 10**-3
l3 = 82.85 * 10**-3
l4 = 73.85 * 10**-3
l5 = 54.57 * 10**-3


# define the unit vectors
ex = np.array([1, 0, 0])
ey = np.array([0, 1, 0])
ez = np.array([0, 0, 1])


# Define the position vectors from i-1 -> i
P01 = (l0 + l1) * ez
P12 = np.zeros(3)
P23 = l2 * ex
P34 = -1*l3 * ez
P45 = np.zeros(3)
P5T = -1*(l4 + l5) * ex

# define the class inputs: rotation axes (H), position vectors (P), and joint_type
H = np.array([ez, -1*ey, -1*ey, -1*ey, -1*ex]).T
P = np.array([P01, P12, P23, P34, P45, P5T]).T
joint_type = [0,0,0,0,0]

 
# define the Robot object
robot = grt.Robot(H, P, joint_type)

# initialize servo arm
Arm = Arm_Device()

# Set the bot to be vertical
#Arm.Arm_serial_servo_write6(0,90,10,45,90,15,1100)
start = np.array([90,90,90,90,90,15])
move_read.move_bot(start)
# THE SERVO ANGLE (the 6th variable) CAN NOT BE ZERO OR IT WILL BREAK THE SERVO
# It must be between 5 and 180

# initialize joint angles, radians
q0 = np.array([90,65,5,70,90])*math.pi/180

print("Initial Joints:\n",q0)
# and compute path
p = grt.fwdkin(robot,q0)
print("Expected Transform:\n",p)


    #End position from FK
block_1 = np.array([90,70,5,55,90,15])

#Generating path
q_des = path_generation.gen_joint_space_path(start, block_1, 5)

#Moving arm through path
path_generation.path_follow(q_des,1.5,5)
    
print(move_read.read_joints())

#Grabbing block
Arm.Arm_serial_servo_write(6, 155, 400)


# desired rotation matrix
Rf = np.array([[-0.8192,0.5736,0],[0,0,1],[.5736,.8192,0]])
# desired position matrix
Pf = [0.1868,0,0.1281]
# make desired transformation
Tf = grt.Transform(Rf,Pf)
# and calculate inverse kinematics
#DO NOT OVERWRITE QF (DUMPER LOCATION)
max_steps=200
qf = grt_ik.iterative_invkin(robot,Tf,q0,max_steps)
while(qf[0]!=True): #if the inverse kinematics does not converge
    max_steps*=2
    qf = grt_ik.iterative_invkin(robot,Tf,q0,max_steps)
print("Necessary Joint Angles:\n",np.array(qf[1][0]))


#Dump Block
start_garb_place = move_read.read_joints()
end_garb_place = np.append(np.array(qf[1][0])*180/math.pi, move_read.read_joints()[5])

steps = 10
time = 1.5
kp = .1
ki = .1
kd = .01

q_des = path_generation.gen_joint_space_path(start_garb_place, end_garb_place, steps)
print(q_des)

error = path_generation.pid_path_follow(q_des, time, steps, kp, ki, kd)
print(error)

Arm.Arm_serial_servo_write(6, 125, 400)
print('dropped')


# initialize joint angles
q2 = np.array([45,70,10,40,90])

#print("Initial Joints:\n",q2)
# and compute path
p = grt.fwdkin(robot,q2)
#print("Expected Transform:\n",p)

#End position from FK
start2 = move_read.read_joints()
end2 = np.array([45,70,10,40,90,15])

#Generating path
q_des2 = path_generation.gen_joint_space_path(start2, end2, 5)

#Moving arm through path
path_generation.path_follow(q_des2,2,5)
    
print(move_read.read_joints())

#Grabbing block
Arm.Arm_serial_servo_write(6, 155, 400)

#Path Following

start_garb_place = move_read.read_joints()
end_garb_place = np.append(np.array(qf[1][0])*180/math.pi, move_read.read_joints()[5])

steps = 10
time = 1.5
kp = .1
ki = .1
kd = .01

q_des = path_generation.gen_joint_space_path(start_garb_place, end_garb_place, steps)
print(q_des)

error = path_generation.pid_path_follow(q_des, time, steps, kp, ki, kd)

print(error)





Initial Joints:
 [1.57079633 1.13446401 0.08726646 1.22173048 1.57079633]
Expected Transform:
 R = [[-0.     -0.     -1.    ]
     [-0.766   0.6428  0.    ]
     [ 0.6428  0.766  -0.    ]]
p = [0.     0.2112 0.0687]


[90. 70.  5. 56. 90. 14.]
Necessary Joint Angles:
 [1.89204041e-04 1.57059102e+00 1.75007333e-01 7.85128857e-01
 1.57071721e+00]
[array([89., 70.,  5., 56., 90., 90.]), array([80.10108406, 71.99882369,  5.50271815, 54.89845699, 89.99954668,
       91.1       ]), array([71.20216812, 73.99764739,  6.00543631, 53.79691397, 89.99909335,
       92.2       ]), array([62.30325218, 75.99647108,  6.50815446, 52.69537096, 89.99864003,
       93.3       ]), array([53.40433624, 77.99529478,  7.01087262, 51.59382794, 89.99818671,
       94.4       ]), array([44.5054203 , 79.99411847,  7.51359077, 50.49228493, 89.99773338,
       95.5       ]), array([35.60650436, 81.99294217,  8.01630892, 49.39074192, 89.99728006,
       96.6       ]), array([26.70758842, 83.99176586,  8.51902708, 48.

4. Perform Inverse Kinematics

In [10]:
# desired rotation matrix
Rf = np.array([[-0.8192,0.5736,0],[0,0,1],[.5736,.8192,0]])
# desired position matrix
Pf = [0.1868,0,0.1281]
# make desired transformation
Tf = grt.Transform(Rf,Pf)
# and calculate inverse kinematics
#DO NOT OVERWRITE QF (DUMPER LOCATION)
max_steps=200
qf = grt_ik.iterative_invkin(robot,Tf,q0,max_steps)
while(qf[0]!=True): #if the inverse kinematics does not converge
    max_steps*=2
    qf = grt_ik.iterative_invkin(robot,Tf,q0,max_steps)
print("Necessary Joint Angles:\n",np.array(qf[1][0]))


Necessary Joint Angles:
 [1.89204041e-04 1.57059102e+00 1.75007333e-01 7.85128857e-01
 1.57071721e+00]


In [11]:
#Path Following

start_garb_place = move_read.read_joints()
end_garb_place = np.append(np.array(qf[1][0])*180/math.pi, move_read.read_joints()[5])

steps = 10
time = 1.5
kp = .9
ki = .3
kd = .01

q_des = path_generation.gen_joint_space_path(start_garb_place, end_garb_place, steps)
print(q_des)

error = path_generation.pid_path_follow(q_des, time, steps, kp, ki, kd)

print(error)

print(move_read.read_joints())

[array([ 89.,  70.,   5.,  56.,  90., 154.]), array([ 80.10108406,  71.99882369,   5.50271815,  54.89845699,
        89.99954668, 154.        ]), array([ 71.20216812,  73.99764739,   6.00543631,  53.79691397,
        89.99909335, 154.        ]), array([ 62.30325218,  75.99647108,   6.50815446,  52.69537096,
        89.99864003, 154.        ]), array([ 53.40433624,  77.99529478,   7.01087262,  51.59382794,
        89.99818671, 154.        ]), array([ 44.5054203 ,  79.99411847,   7.51359077,  50.49228493,
        89.99773338, 154.        ]), array([ 35.60650436,  81.99294217,   8.01630892,  49.39074192,
        89.99728006, 154.        ]), array([ 26.70758842,  83.99176586,   8.51902708,  48.2891989 ,
        89.99682674, 154.        ]), array([ 17.80867247,  85.99058956,   9.02174523,  47.18765589,
        89.99637341, 154.        ]), array([  8.90975653,  87.98941325,   9.52446339,  46.08611287,
        89.99592009, 154.        ]), array([1.08405930e-02, 8.99882369e+01, 1.00271815e+01,

In [7]:
#Path Following

start_garb_place = move_read.read_joints()
end_garb_place = np.append(np.array(qf[1][0])*180/math.pi, move_read.read_joints()[5])

kp = 1.2

q_des = path_generation.gen_joint_space_path(start_garb_place, end_garb_place, 5)

for i in range(len(q_des)):

        move_read.move_bot(q_des[i])

        q_act = move_read.read_joints()

        error = q_des[i] - q_act

        print(error)

        er = max(abs(error))

        while abs(er) > 1:

            print('hi')

            q_command = q_des[i] + kp * error

            move_read.move_bot(q_command)

            q_act_new = move_read.read_joints()

            error = q_des[i] - q_act_new

            er = max(abs(error))

        print(error)
print('done')

[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]
[ 0.00216812 -0.00235261 -0.19456369 -0.00308603 -0.00090665  0.        ]
[ 0.00216812 -0.00235261 -0.19456369 -0.00308603 -0.00090665  0.        ]
[ 0.00433624 -0.00470522 -0.38912738 -0.00617206 -0.00181329  0.        ]
[ 0.00433624 -0.00470522 -0.38912738 -0.00617206 -0.00181329  0.        ]
[ 0.00650436 -0.00705783 -0.58369108 -0.00925808 -0.00271994  0.        ]
[ 0.00650436 -0.00705783 -0.58369108 -0.00925808 -0.00271994  0.        ]
[ 0.00867247 -0.00941044 -0.77825477 -0.01234411 -0.00362659  0.        ]
[ 0.00867247 -0.00941044 -0.77825477 -0.01234411 -0.00362659  0.        ]
[ 0.01084059 -0.01176305 -0.97281846 -0.01543014 -0.00453323  0.        ]
[ 0.01084059 -0.01176305 -0.97281846 -0.01543014 -0.00453323  0.        ]
done


In [12]:
#Releasing block
Arm.Arm_serial_servo_write(6, 125, 400)

Block 2

In [13]:
# initialize joint angles, radians
q2 = np.array([45,70,10,40,90])

print("Initial Joints:\n",q2)
# and compute path
p = grt.fwdkin(robot,q2)
print("Expected Transform:\n",p)

Initial Joints:
 [45 70 10 40 90]
Expected Transform:
 R = [[ 0.4277  0.6539 -0.624 ]
     [ 0.6928  0.2063  0.691 ]
     [ 0.5806 -0.7279 -0.3648]]
p = [-0.0706 -0.1144  0.1032]




In [14]:
#End position from FK
start2 = move_read.read_joints()
end2 = np.array([45,70,10,40,90,15])

#Generating path
q_des2 = path_generation.gen_joint_space_path(start2, end2, 5)

#Moving arm through path
path_generation.path_follow(q_des2,2,5)
    
print(move_read.read_joints())

#Grabbing block
Arm.Arm_serial_servo_write(6, 155, 400)

[44. 70. 11. 41. 90. 15.]


In [15]:
#Path Following

start_garb_place = move_read.read_joints()
end_garb_place = np.append(np.array(qf[1][0])*180/math.pi, move_read.read_joints()[5])

kp = 1.2

q_des = path_generation.gen_joint_space_path(start_garb_place, end_garb_place, 5)

for i in range(len(q_des)):

        move_read.move_bot(q_des[i])

        q_act = move_read.read_joints()

        error = q_des[i] - q_act

        print(error)

        er = max(abs(error))

        while abs(er) > 1:

            print('hi')

            q_command = q_des[i] + kp * error

            move_read.move_bot(q_command)

            q_act_new = move_read.read_joints()

            error = q_des[i] - q_act_new

            er = max(abs(error))

        print(error)
print('done')

[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]
[ 2.02168119e-01  9.97647390e-01 -1.94563692e-01  7.96913972e-01
 -9.06646999e-04  0.00000000e+00]
[ 2.02168119e-01  9.97647390e-01 -1.94563692e-01  7.96913972e-01
 -9.06646999e-04  0.00000000e+00]
[ 0.40433624  0.99529478 -0.38912738  0.59382794 -0.00181329  1.        ]
[ 0.40433624  0.99529478 -0.38912738  0.59382794 -0.00181329  1.        ]
[-0.39349564  0.99294217 -0.58369108  0.39074192 -0.00271994  1.        ]
[-0.39349564  0.99294217 -0.58369108  0.39074192 -0.00271994  1.        ]
[-0.19132753         nan -0.77825477  0.18765589 -0.00362659  1.        ]
[-0.19132753         nan -0.77825477  0.18765589 -0.00362659  1.        ]
[ 0.01084059  0.98823695 -0.97281846  0.98456986 -0.00453323  1.        ]
[ 0.01084059  0.98823695 -0.97281846  0.98456986 -0.00453323  1.        ]
done


In [16]:
#Releasing block
Arm.Arm_serial_servo_write(6, 125, 400)

BLOCK 3

In [12]:
# initialize joint angles, radians
q3 = np.array([135,70,10,40,90,15])

print("Initial Joints:\n",q3)
# and compute path
p = grt.fwdkin(robot,q3)
print("Expected Transform:\n",p)

Initial Joints:
 [135  70  10  40  90  15]
Expected Transform:
 R = [[-0.811  -0.4774 -0.3381]
     [ 0.0719  0.4922 -0.8675]
     [ 0.5806 -0.7279 -0.3648]]
p = [ 0.1339 -0.0119  0.1032]




In [13]:
#End position from FK
start3 = move_read.read_joints()
end3 = np.array([135,70,10,40,90,15])

#Generating path
q_des3 = path_generation.gen_joint_space_path(start3, end3, 5)

#Moving arm through path
path_generation.path_follow(q_des3,2,5)
    
print(move_read.read_joints())

#Grabbing block
Arm.Arm_serial_servo_write(6, 155, 400)

[134.  70.  10.  41.  90.  17.]


In [14]:
#Path Following

start_garb_place = move_read.read_joints()
end_garb_place = np.append(np.array(qf[1][0])*180/math.pi, move_read.read_joints()[5])

kp = 1.2

q_des = path_generation.gen_joint_space_path(start_garb_place, end_garb_place, 5)

for i in range(len(q_des)):

        move_read.move_bot(q_des[i])

        q_act = move_read.read_joints()

        error = q_des[i] - q_act

        print(error)

        er = max(abs(error))

        while abs(er) > 1:

            print('hi')

            q_command = q_des[i] + kp * error

            move_read.move_bot(q_command)

            q_act_new = move_read.read_joints()

            error = q_des[i] - q_act_new

            er = max(abs(error))

        print(error)
print('done')

[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]
[0.20216812 1.99764739 0.00543631 0.79691397 0.99909335 0.        ]
hi
[ 0.20216812 -0.00235261  0.00543631 -0.20308603 -0.00090665  0.        ]
[ 4.04336237e-01  1.99529478e+00  1.08726161e-02  5.93827944e-01
 -1.81329400e-03  0.00000000e+00]
hi
hi
[ 0.40433624 -0.00470522  0.01087262 -0.40617206 -0.00181329  0.        ]
[-0.39349564  1.99294217  0.01630892  0.39074192 -0.00271994  0.        ]
hi
hi
hi
[-0.39349564 -0.00705783  0.01630892  0.39074192 -0.00271994  0.        ]
[-0.19132753  1.99058956  0.02174523  0.18765589 -0.00362659  0.        ]
hi
hi
hi
[-0.19132753  0.99058956  0.02174523  0.18765589 -0.00362659  1.        ]
[ 0.01084059  1.98823695  0.02718154  0.98456986 -0.00453323  1.        ]
hi
hi
[ 0.01084059 -0.01176305  0.02718154 -0.01543014 -0.00453323  1.        ]
done


In [15]:
#Releasing block
Arm.Arm_serial_servo_write(6, 125, 400)

BLOCK 4

In [16]:
# initialize joint angles, radians
q4 = np.array([65,25,60,60,90,15])
move_read.move_bot(q4)

print("Initial Joints:\n",q4)
# and compute path
p = grt.fwdkin(robot,q4)
print("Expected Transform:\n",p)

Initial Joints:
 [65 25 60 60 90 15]
Expected Transform:
 R = [[-0.4971  0.1353 -0.8571]
     [ 0.7308  0.5978 -0.3295]
     [ 0.4677 -0.7902 -0.396 ]]
p = [ 0.0259 -0.038   0.115 ]




In [17]:
#End position from FK
start4 = move_read.read_joints()
end4 = np.array([65,25,60,60,90,15])

#Generating path
q_des4 = path_generation.gen_joint_space_path(start4, end4, 5)

#Moving arm through path
path_generation.path_follow(q_des4,2,5)
    
print(move_read.read_joints())

#Grabbing block
Arm.Arm_serial_servo_write(6, 155, 400)

[65. 23. 59. 59. 90. 17.]


In [18]:
#Path Following

start_garb_place = move_read.read_joints()
end_garb_place = np.append(np.array(qf[1][0])*180/math.pi, move_read.read_joints()[5])

kp = 1.2

q_des = path_generation.gen_joint_space_path(start_garb_place, end_garb_place, 5)

for i in range(len(q_des)):

        move_read.move_bot(q_des[i])

        q_act = move_read.read_joints()

        error = q_des[i] - q_act

        print(error)

        er = max(abs(error))

        while abs(er) > 1:

            print('hi')

            q_command = q_des[i] + kp * error

            move_read.move_bot(q_command)

            q_act_new = move_read.read_joints()

            error = q_des[i] - q_act_new

            er = max(abs(error))

        print(error)
print('done')

[ 0.  0.  0. -1.  0.  0.]
[ 0.  0.  0. -1.  0.  0.]
[ 2.16811860e-03  2.39764739e+00 -7.94563692e-01 -1.80308603e+00
 -9.06646999e-04  0.00000000e+00]
hi
hi
[ 2.16811860e-03 -6.02352610e-01  2.05436308e-01  1.96913972e-01
 -9.06646999e-04  1.00000000e+00]
[ 4.33623721e-03  1.79529478e+00 -5.89127384e-01 -2.60617206e+00
 -1.81329400e-03  0.00000000e+00]
hi
hi
hi
[ 0.00433624  0.79529478  0.41087262  0.39382794 -0.00181329  1.        ]
[ 0.00650436  2.19294217 -0.38369108 -1.40925808 -0.00271994  1.        ]
hi
[ 0.00650436 -0.80705783  0.61630892  0.59074192 -0.00271994  1.        ]
[ 0.00867247  1.59058956 -0.17825477 -1.21234411 -0.00362659  1.        ]
hi
[ 0.00867247  0.59058956 -0.17825477  0.78765589 -0.00362659  1.        ]
[ 0.01084059  1.98823695  0.02718154 -1.01543014 -0.00453323  1.        ]
hi
hi
hi
[ 0.01084059  0.98823695  0.02718154 -0.01543014 -0.00453323  1.        ]
done


In [19]:
#Releasing block
Arm.Arm_serial_servo_write(6, 125, 400)

BLOCK 5

In [20]:
# initialize joint angles, radians
q5 = np.array([115,25,60,60,90,15])

print("Initial Joints:\n",q4)
# and compute path
p = grt.fwdkin(robot,q4)
print("Expected Transform:\n",p)

Initial Joints:
 [65 25 60 60 90 15]
Expected Transform:
 R = [[-0.4971  0.1353 -0.8571]
     [ 0.7308  0.5978 -0.3295]
     [ 0.4677 -0.7902 -0.396 ]]
p = [ 0.0259 -0.038   0.115 ]




In [21]:
#End position from FK
start5 = move_read.read_joints()
end5 = np.array([115,25,60,60,90,15])

#Generating path
q_des5 = path_generation.gen_joint_space_path(start5, end5, 5)

#Moving arm through path
path_generation.path_follow(q_des5,2,5)
    
print(move_read.read_joints())

#Grabbing block
Arm.Arm_serial_servo_write(6, 155, 400)

[114.  25.  59.  59.  90.  17.]


In [22]:
start_garb_place = move_read.read_joints()
end_garb_place = np.append(np.array(qf[1][0])*180/math.pi, move_read.read_joints()[5])

kp = 1.2

q_des = path_generation.gen_joint_space_path(start_garb_place, end_garb_place, 5)

for i in range(len(q_des)):

        move_read.move_bot(q_des[i])

        q_act = move_read.read_joints()

        error = q_des[i] - q_act

        print(error)

        er = max(abs(error))

        while abs(er) > 1:

            print('hi')

            q_command = q_des[i] + kp * error

            move_read.move_bot(q_command)

            q_act_new = move_read.read_joints()

            error = q_des[i] - q_act_new

            er = max(abs(error))

        print(error)
print('done')

[0. 1. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0.]
[ 2.02168119e-01  2.19764739e+00 -7.94563692e-01 -1.80308603e+00
 -9.06646999e-04  0.00000000e+00]
hi
hi
[ 0.20216812  0.19764739  0.20543631  0.19691397 -0.00090665  0.        ]
[ 4.04336237e-01  2.39529478e+00 -5.89127384e-01 -2.60617206e+00
 -1.81329400e-03  0.00000000e+00]
hi
hi
hi
[ 0.40433624 -0.60470522  0.41087262 -0.60617206 -0.00181329  0.        ]
[ 0.60650436  1.59294217 -0.38369108 -1.40925808 -0.00271994  0.        ]
hi
[ 0.60650436  0.59294217  0.61630892  0.59074192 -0.00271994  0.        ]
[-0.19132753  1.79058956 -0.17825477 -1.21234411 -0.00362659  0.        ]
hi
[ 0.80867247 -0.20941044 -0.17825477  0.78765589 -0.00362659  1.        ]
[ 0.01084059  0.98823695  0.02718154 -1.01543014 -0.00453323  1.        ]
hi
[ 0.01084059 -0.01176305  0.02718154 -0.01543014 -0.00453323  1.        ]
done


In [23]:
#Releasing block
Arm.Arm_serial_servo_write(6, 125, 400)